In [1]:
import nest_asyncio
nest_asyncio.apply()

import sys
sys.path.append("../")

import asyncio
import time
from typing import List, Optional, Dict
from pydantic import (
    Field,
    BaseModel,
    validator,
    ConfigDict,
    field_validator,
    field_serializer,
)
from langchain_core.runnables import RunnableParallel

from desci_sense.configs import default_init_parser_config
from desci_sense.shared_functions.dataloaders import scrape_post
from desci_sense.shared_functions.schema.post import RefPost
from desci_sense.shared_functions.parsers.firebase_api_parser import FirebaseAPIParser, PromptCase
from desci_sense.shared_functions.parsers.multi_chain_parser import MultiChainParser
from desci_sense.shared_functions.web_extractors.metadata_extractors import (
    RefMetadata,
)
from langchain_core.runnables import RunnableParallel

In [2]:
config = default_init_parser_config(semantics_model="mistralai/mistral-7b-instruct",
                                    kw_model="mistralai/mistral-7b-instruct")
config["general"]["ref_metadata_method"] = "citoid"

In [3]:
parser = FirebaseAPIParser(config=config)

2024-04-02 13:05:05.127 | INFO     | desci_sense.shared_functions.parsers.firebase_api_parser:set_md_extract_method:164 - Setting metadata extraction method to citoid...
2024-04-02 13:05:05.128 | INFO     | desci_sense.shared_functions.parsers.firebase_api_parser:__init__:120 - Loading parser model (type=mistralai/mistral-7b-instruct)...
/Users/ronentamari/anaconda3/envs/asensebot/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
2024-04-02 13:05:05.395 | INFO     | desci_sense.shared_functions.parsers.firebase_api_parser:set_kw_md_extract_method:168 - Setting keywords metadata extract

In [5]:
chain_dict = {
    "kw_1": parser.kw_extraction["chain"],
    "kw_2": parser.kw_extraction["chain"],
}


In [10]:
kw_input = {"kw_input": "This post is about crypto, web3 and AI."}

In [8]:
parallel_chain = RunnableParallel(**chain_dict)

In [13]:
res = parallel_chain.invoke(kw_input)

In [14]:
res

{'kw_1': {'reasoning': '[Reasoning Steps]\n\n[System error: failed to extract reasoning steps since the generated output was in an invalid format]\n\n[Candidate Keywords]\n\n[System error: failed to extract candidate keywords since the generated output was in an invalid format.]',
  'final_answer': '<error>',
  'valid_keywords': [],
  'academic_kw': 'not-detected',
  'raw_text': "As a topic of discussion, it's interesting to consider the intersection of cryptocurrency, Web3, and artificial intelligence. Cryptocurrency is a decentralized digital currency that uses cryptography for security and is distributed through a network of computers. Web3, on the other hand, is the next evolution of the internet, which is built on decentralized technologies like blockchain and smart contracts. And finally, AI is the development of computer systems that can perform tasks that would normally require human intelligence, such as visual perception, speech recognition, decision-making, and language tran